# Spotify API 
Downloading the data and preparing it for the sake of the analysis

In [4]:
# pip install spotipy

Note: you may need to restart the kernel to use updated packages.


We have decided to generate the dataset ourselves by leveraging the Spotify API and the Spotify library in python to extract and analyze the data 

Created own developer spotify account (API) 

Extracted to spotify generated playlist à extracted random samples 
(Spotify has created over 1400 playlists so far) 

In [37]:
# Importing the libraries
import sys
import spotipy
import spotipy.util as util
import pandas as pd
from spotipy.oauth2 import SpotifyClientCredentials
import itertools
client_credentials_manager = SpotifyClientCredentials('40cd9cd27c7c4689bc36774f5aac188b','10d3a4b01aea4976ac89c831db901a6d')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=20)

 Retrieving a sample of artists large enough from the artists available on the app. We will be referring to playlists created by Spotify it self and not the ones created by the users. These playlists are a good reference since they will be covering as many genres available as possible. The following command will collect these playlists.

In [38]:
playlists = sp.user_playlists('spotify')
playlist_ids = []
while playlists:
    for i, playlist in enumerate(playlists['items']):
        playlist_ids.append(playlist['id'])
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

Gathering the data related to the tracks found in each of the playlists retrieved. The data is inclusive of data related to the artists as well. Duplicate artists problem will be handled and we will transform these information into a dataframe.

In [42]:
trackslist = [] 
for j,i in enumerate(playlist_ids):
#     if j%50 == 0:
#         print(j)
    tracks = sp.playlist_tracks(i, limit = 100)
    trackslist.append(tracks['items'])
    while tracks['next']:
        tracks = sp.next(tracks)
        trackslist.append(tracks['items'])

In [45]:
tracks = [j['track'] for i in trackslist for j in i]
artistids = [j['id'] for i in tracks if i!=None for j in i['artists']]
artistids = pd.Series(artistids).drop_duplicates()
artistids = artistids.dropna()
artists = sp.artists(artistids[:50])['artists']
for i in range(50,len(artistids),50):
    artists.append(sp.artists(artistids[i:(i+50)])['artists'])
artists = artists[:50] + [j for i in artists[50:] for j in i]

Returning up to 20 related artists for each artist passed as input. An Edge is created between artists if these show up as related. Creating a network of artists!

In [46]:
related_artists = []
for j, i in enumerate(artistids):
    related_artists.append(sp.artist_related_artists(i))
related_artists = [i for i in related_artists if i!= None]
related_artists = [i['artists'] for i in related_artists]
len(related_artists)

37281

In [48]:
relart = related_artists
for i in range(len(relart)):
    if relart[i]!=[]:
        for j in range(len(relart[i])):
            relart[i][j].update(source_id = artistids.iloc[i])
relart = [j for i in relart for j in i]
for i in range(len(relart)):
    relart[i]['followers'] = relart[i]['followers']['total']
relart = pd.DataFrame.from_dict(relart)

In [49]:
artists=pd.DataFrame(artists)

In [50]:
all_artist_data = pd.merge(artists, relart, left_on = 'id', right_on = 'source_id',suffixes=('_source', '_related'))
all_artist_data = all_artist_data.drop_duplicates(subset = ['source_id','id_related'])
all_artist_data = all_artist_data[['followers_source','genres_source','id_source','name_source','popularity_source',
                                  'uri_source','followers_related', 'genres_related','id_related', 'name_related',
                                   'popularity_related','uri_related']]
all_artist_data.to_csv('all_artist_data_final.csv')

In [51]:

artists = artists[[ 'followers', 'genres','id','name','popularity','uri']]
artists.to_csv('artist_data_final.csv')
all_artist_data.head()

,followers_source,genres_source,id_source,name_source,popularity_source,uri_source,followers_related,genres_related,id_related,name_related,popularity_related,uri_related
0,"{'href': None, 'total': 34336338}","[dance pop, pop, pop dance, post-teen pop]",06HL4z0CvFAxyc27GXpf02,Taylor Swift,93,spotify:artist:06HL4z0CvFAxyc27GXpf02,23742682,"[dance pop, pop, pop dance, post-teen pop]",0C8ZW7ezQVs4URX5aX7Kqx,Selena Gomez,86,spotify:artist:0C8ZW7ezQVs4URX5aX7Kqx
1,"{'href': None, 'total': 34336338}","[dance pop, pop, pop dance, post-teen pop]",06HL4z0CvFAxyc27GXpf02,Taylor Swift,93,spotify:artist:06HL4z0CvFAxyc27GXpf02,13426581,"[dance pop, pop, pop dance, post-teen pop]",5YGY8feqx7naU7z4HrwZM6,Miley Cyrus,91,spotify:artist:5YGY8feqx7naU7z4HrwZM6
2,"{'href': None, 'total': 34336338}","[dance pop, pop, pop dance, post-teen pop]",06HL4z0CvFAxyc27GXpf02,Taylor Swift,93,spotify:artist:06HL4z0CvFAxyc27GXpf02,18533992,"[dance pop, pop, pop dance, post-teen pop]",6S2OmqARrzebs0tKUEyXyp,Demi Lovato,84,spotify:artist:6S2OmqARrzebs0tKUEyXyp
3,"{'href': None, 'total': 34336338}","[dance pop, pop, pop dance, post-teen pop]",06HL4z0CvFAxyc27GXpf02,Taylor Swift,93,spotify:artist:06HL4z0CvFAxyc27GXpf02,9784381,"[dance pop, hip pop, pop, pop dance, post-teen...",6JL8zeS1NmiOftqZTRgdTz,Meghan Trainor,82,spotify:artist:6JL8zeS1NmiOftqZTRgdTz
4,"{'href': None, 'total': 34336338}","[dance pop, pop, pop dance, post-teen pop]",06HL4z0CvFAxyc27GXpf02,Taylor Swift,93,spotify:artist:06HL4z0CvFAxyc27GXpf02,40210307,"[canadian pop, pop, post-teen pop]",1uNFoZAHBGtllmzznpCI3s,Justin Bieber,96,spotify:artist:1uNFoZAHBGtllmzznpCI3s


In [52]:
relart  = relart[[ 'followers', 'genres','id','name','popularity','uri']]
allart = relart.append(artists)
allart = allart.drop_duplicates(subset = ['id'])


print(allart.head())
allart.columns = ['followers', 'genres','spotify id','Label','popularity','uri']
strgnrs = ['_'.join(i) for i in allart['genres']]
allart['strgnrs'] = strgnrs

  followers                                             genres  \
0  23742682         [dance pop, pop, pop dance, post-teen pop]   
1  13426581         [dance pop, pop, pop dance, post-teen pop]   
2  18533992         [dance pop, pop, pop dance, post-teen pop]   
3   9784381  [dance pop, hip pop, pop, pop dance, post-teen...   
4  40210307                 [canadian pop, pop, post-teen pop]   

                       id            name  popularity  \
0  0C8ZW7ezQVs4URX5aX7Kqx    Selena Gomez          86   
1  5YGY8feqx7naU7z4HrwZM6     Miley Cyrus          91   
2  6S2OmqARrzebs0tKUEyXyp     Demi Lovato          84   
3  6JL8zeS1NmiOftqZTRgdTz  Meghan Trainor          82   
4  1uNFoZAHBGtllmzznpCI3s   Justin Bieber          96   

                                     uri  
0  spotify:artist:0C8ZW7ezQVs4URX5aX7Kqx  
1  spotify:artist:5YGY8feqx7naU7z4HrwZM6  
2  spotify:artist:6S2OmqARrzebs0tKUEyXyp  
3  spotify:artist:6JL8zeS1NmiOftqZTRgdTz  
4  spotify:artist:1uNFoZAHBGtllmzznpCI3s  


In [53]:
allart.to_csv('artists_nodes_gephi.csv', index = False)

In [54]:
all_artist_data.columns = ['followers_source','genres_source','id_source','Source','popularity_source',
                                  'uri_source','followers_related', 'genres_related','id_related', 'Target',
                                   'popularity_related','uri_related']
all_artist_data.to_csv('artists_edges_gephi.csv', index = False)

Making Subset of the artist data that fall under some genre examples

In [55]:
# Artists falling that were tagged With 'EDM' Genre would be included in this dataset
directed_artist = all_artist_data.reset_index(drop = True)
source_mask = directed_artist['genres_source'].apply(lambda x: 'edm' in x)
edm = directed_artist.loc[source_mask]
edm.to_csv('edm.csv',index = False)
directed_artist = allart.reset_index(drop = True)
source_mask = directed_artist['genres'].apply(lambda x: 'edm' in x)
edmnodes = directed_artist.loc[source_mask]
edmnodes['Id'] = edmnodes['Label'].copy()
edmnodes.to_csv('edm_source_nodes.csv',index = False)

<ipython-input-55-59f2cf041a0c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  edmnodes['Id'] = edmnodes['Label'].copy()


In [57]:
# Artists falling that were tagged With 'Latin' Genre would be included in this dataset
directed_artist = all_artist_data.reset_index(drop = True)
source_mask = directed_artist['genres_source'].apply(lambda x: 'latin' in x)
latin = directed_artist.loc[source_mask]
latin.to_csv('latin.csv',index = False)
directed_artist = allart.reset_index(drop = True)
source_mask = directed_artist['genres'].apply(lambda x: 'latin' in x)
latinnodes = directed_artist.loc[source_mask]
latinnodes['Id'] = latinnodes['Label'].copy()
latinnodes.to_csv('latin_source_nodes.csv',index = False)

<ipython-input-57-730c3c18b098>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  latinnodes['Id'] = latinnodes['Label'].copy()


In [58]:
# Artists falling that were tagged With 'Tropical House' Genre would be included in this dataset
directed_artist = all_artist_data.reset_index(drop = True)
source_mask = directed_artist['genres_source'].apply(lambda x: 'tropical house' in x)
tropic_house = directed_artist.loc[source_mask]
tropic_house.to_csv('tropic_house.csv',index = False)
directed_artist = allart.reset_index(drop = True)
source_mask = directed_artist['genres'].apply(lambda x: 'tropic house' in x)
tropic_housenodes = directed_artist.loc[source_mask]
tropic_housenodes['Id'] = tropic_housenodes['Label'].copy()
tropic_housenodes.to_csv('tropic_house_source_nodes.csv',index = False)

In [59]:
# Artists falling that were tagged With 'Bachata' Genre would be included in this dataset
directed_artist = all_artist_data.reset_index(drop = True)
source_mask = directed_artist['genres_source'].apply(lambda x: 'bachata' in x)
bachata = directed_artist.loc[source_mask]
bachata.to_csv('bachata.csv',index = False)
directed_artist = allart.reset_index(drop = True)
source_mask = directed_artist['genres'].apply(lambda x: 'bachata' in x)
bachatanodes = directed_artist.loc[source_mask]
bachatanodes['Id'] = bachatanodes['Label'].copy()
bachatanodes.to_csv('bachata_source_nodes.csv',index = False)

<ipython-input-59-ea405b3cc82f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bachatanodes['Id'] = bachatanodes['Label'].copy()


In [60]:
# Artists falling that were tagged With 'German HipHop' Genre would be included in this dataset
directed_artist = all_artist_data.reset_index(drop = True)
source_mask = directed_artist['genres_source'].apply(lambda x: 'german hip hop' in x)
german_hiphop = directed_artist.loc[source_mask]
german_hiphop.to_csv('german_hiphop.csv',index = False)
directed_artist = allart.reset_index(drop = True)
source_mask = directed_artist['genres'].apply(lambda x: 'german hip hop' in x)
german_hiphopnodes = directed_artist.loc[source_mask]
german_hiphopnodes['Id'] = german_hiphopnodes['Label'].copy()
german_hiphopnodes.to_csv('german_hiphop_source_nodes.csv',index = False)

<ipython-input-60-d37d87667245>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  german_hiphopnodes['Id'] = german_hiphopnodes['Label'].copy()


In [61]:
# Artists falling that were tagged With 'HipHop' Genre would be included in this dataset
directed_artist = all_artist_data.reset_index(drop = True)
source_mask = directed_artist['genres_source'].apply(lambda x: 'hip hop' in x)
hiphop = directed_artist.loc[source_mask]
hiphop.to_csv('hiphop.csv',index = False)
directed_artist = allart.reset_index(drop = True)
source_mask = directed_artist['genres'].apply(lambda x: 'hip hop' in x)
hiphopnodes = directed_artist.loc[source_mask]
hiphopnodes['Id'] = hiphopnodes['Label'].copy()
hiphopnodes.to_csv('hiphop_source_nodes.csv',index = False)

<ipython-input-61-879e3f03a37f>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hiphopnodes['Id'] = hiphopnodes['Label'].copy()
